Ceci n'est pas un commentaire


### Imports

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Sources
src_url = "/kaggle/input/test-dataset/Google_Recaptcha_V2_Images_Dataset/images/"

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mikhailma/test-dataset")

print("Path to dataset files:", path)

100%|██████████| 393M/393M [00:03<00:00, 103MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mikhailma/test-dataset/versions/1


In [ ]:
import os
os.listdir(path)

['Google_Recaptcha_V2_Images_Dataset']

In [ ]:
def build_model(dense_neurons):
    model = tf.keras.Sequential([
        layers.InputLayer((28,28,1)),
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(dense_neurons, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
model = build_model(dense_neurons=64)
BAT = 64
    # Train the model
history = model.fit(
    train_images,train_labels,
    batch_size=BAT,
    epochs=5,
    verbose=2
)
# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels, verbose=2)

How to load images with tensorflow  : https://www.tensorflow.org/api_docs/python/tf/keras/utils/load_img

In [ ]:
tf.keras.utils.load_img(
    path,
    color_mode='rgb',
    target_size=None,
    interpolation='nearest',
    keep_aspect_ratio=False
)